In [1]:
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
from konlpy.tag import Okt
import logging
import time
import re
import os
import json
from tqdm import tqdm
import torch
import os
import json
import unicodedata
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from pdf2image import convert_from_path
import pytesseract
# Vector stores
import fitz  # PyMuPDF
import pymupdf4llm
from pymupdf4llm import to_markdown
from PyPDF2 import PdfReader

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cuda' if torch.cuda.is_available() else device  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

from langchain_unstructured import UnstructuredLoader

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text


mps


### markdown, table

In [2]:
markdown_headers = [
    ("#", "Header1"),
    ("##", "Header2"),
    ("###", "Header3"),
    ("####", "Header4"),
    ("#####", "Header5")
]
def extract_headers(text):
    headers = {}
    lines = text.splitlines()

    for line in lines:
        for markdown, header_name in markdown_headers:
            if line.startswith(markdown):
                header_text = line.replace(markdown, "").strip()
                headers[header_name] = header_text
                break
    return headers
# 문서의 메타데이터로 기존 메타데이터와 헤더 추가
def create_document_with_combined_metadata(text, existing_metadata):
    headers = extract_headers(text)
    combined_metadata = {**existing_metadata, **headers}  # 기존 메타데이터와 새 헤더 병합
    document = Document(page_content=text, metadata=combined_metadata)
    return document


In [37]:
def get_docs(pdf_path):
    documents = []
    try:
        # 페이지 단위로 pymupdf4llm 사용
        md_read = pymupdf4llm.to_markdown(pdf_path,page_chunks=True)
        total_pages = len(md_read)    
        
        for page_data in md_read:
            page_number = page_data.get('metadata', {}).get('page', None)
            text = clean_text(normalize_string(page_data['text']))
            metadata = {
                "Source_path": pdf_path,
                "page_number": page_number,
                "total_pages": total_pages,
                "tables": page_data.get('tables', None),
            }
            
            documents.append(Document(page_content=text, metadata=metadata))
    except Exception as e:
        logging.error(f"Failed to process PDF with pymupdf4llm: {e}")

    return documents

def chunk_documents(docs):
    markdown_headers = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4")
    ]
    
    # 마크다운 스플리터 설정
    markdown_splitter = RecursiveCharacterTextSplitter(
        separators = [
                # First, try to split along Markdown headings (starting with level 2)
                "\n#{1,6} ",
                # Note the alternative syntax for headings (below) is not handled here
                # Heading level 2
                # ---------------
                # End of code block
                "```\n",
                # Horizontal lines
                "\n\\*\\*\\*+\n",
                "\n---+\n",
                "\n___+\n",
                # Note that this splitter doesn't handle horizontal lines defined
                # by *three or more* of ***, ---, or ___, but this is not handled
                "\n\n",
                "\n",
                " ",
                "",
            ],
        chunk_size=800,
        chunk_overlap=80,
    )
    
    chunks = []
    for doc in docs:
        text = doc.page_content
        metadata = doc.metadata
        md_header_splits = markdown_splitter.split_text(text)
        for md_split in md_header_splits:   
            # 테이블이 있는 페이지는 청크로 분할하지 않음
            chunks.append(Document(page_content=md_split, metadata = metadata))
    return chunks

def make_db(df):
    documents = []
    chunks = []
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(get_docs(pdf_file))
        # 청크 생성
        # chunks.extend(chunk_documents(documents))
    print(f"Total number of documents: {len(documents)}")
    
    # print(f"Total number of chunks: {len(chunks)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

### konlptextsplitter, pytessearct

In [4]:
# def ocr_extract_text_from_pdf(pdf_path):
#     documents = []
#     try:
#         # PDF를 이미지로 변환
#         images = convert_from_path(pdf_path, dpi=500)
        
#         # 각 페이지에서 OCR로 텍스트 추출
#         for i, image in enumerate(images):
#             text = pytesseract.image_to_string(image, lang='kor+eng')
#             text = clean_text(normalize_string(text))
#             # 각 페이지마다 별도의 Document 생성
#             documents.append(Document(page_content=text, metadata={"source": pdf_path, "page": i + 1}))
    
#     except Exception as e:
#         print(f"Error extracting text from {pdf_path}: {e}")
    
#     return documents

# def load_pdf_file(pdf_file):
#     try:
#         # OCR을 이용한 텍스트 추출
#         documents = ocr_extract_text_from_pdf(pdf_file)
#         return documents
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []

# def split_documents_with_konlpy(documents):
#     adjusted_documents = []
#     konlpy_splitter = KonlpyTextSplitter(separator='\n\n')

#     for doc in documents:
#         # KonlpyTextSplitter로 문장 단위 분리
#         sentences = konlpy_splitter.split_text(doc.page_content)
#         for sentence in sentences:
#             adjusted_documents.append(Document(page_content=sentence, metadata=doc.metadata))
    
#     return adjusted_documents

# def make_db(df, chunk_name="train", chunk_size=800, chunk_overlap=60):
#     documents = []
#     chunk_file_path = f"{chunk_name}_chunks.json"

#     if os.path.exists(chunk_file_path):
#         print(f"Loading chunks from {chunk_file_path}...")
#         documents = load_chunks(chunk_file_path)
#     else:
#         pdf_files = df['Source_path'].unique()
#         with ThreadPoolExecutor() as executor:
#             results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
#         for result in results:
#             documents.extend(result)
        


#         # 청크 데이터를 저장
#         save_chunks(documents, chunk_file_path)

#     # KonlpyTextSplitter로 문장 단위로 먼저 분할
#     documents = split_documents_with_konlpy(documents)
    
#     # make_db에서 RecursiveCharacterTextSplitter로 청크를 분할
#     recursive_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         separators=["\n\n", "."]
#     )
#     documents = recursive_splitter.split_documents(documents)
    
#     faiss = FAISS.from_documents(documents, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(documents)
    
#     return faiss, bm

# def save_chunks(documents, path):
#     chunk_data = []
#     for doc in documents:
#         chunk_data.append({
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         })
    
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
#     print(f"Chunks saved to {path}")

# def load_chunks(path):
#     with open(path, "r", encoding="utf-8") as f:
#         chunk_data = json.load(f)
    
#     documents = []
#     for chunk in chunk_data:
#         documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
#     print(f"Chunks loaded from {path}")
#     return documents

### 언스트럭처

In [6]:
# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s
# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = UnstructuredLoader(pdf_file,
#                                         chunking_strategy = 'by_title',
#                                         max_characters = 1000,
#                                         overlap = 200)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []

# def make_db(df, chunk_name="train"):
#     documents = []
#     chunk_file_path = f"{chunk_name}_chunks.json"

#     if os.path.exists(chunk_file_path):
#         # 이미 저장된 청크 데이터가 있으면 불러오기
#         print(f"Loading chunks from {chunk_file_path}...")
#         documents = load_chunks(chunk_file_path)
#     else:
#         # 청크 데이터가 없으면 PDF 파일을 처리하여 새로 생성
#         pdf_files = df['Source_path'].unique()

#         # tqdm으로 파일 처리 진행 상황 표시
#         with ThreadPoolExecutor() as executor:
#             results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
        
#         for result in results:
#             documents.extend(result)
        
#         # 정규화
#         for doc in documents:
#             doc.page_content = normalize_string(doc.page_content)
        
#         # 청크 데이터를 저장
#         save_chunks(documents, chunk_file_path)
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(documents, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(documents)
    
#     return faiss, bm

# import json

# def save_chunks(documents, path):
#     chunk_data = []
#     for doc in documents:
#         chunk_data.append({
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         })
    
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(chunk_data, f, ensure_ascii=False, indent=4)
    
#     print(f"Chunks saved to {path}")

# from langchain.docstore.document import Document

# def load_chunks(path):
#     with open(path, "r", encoding="utf-8") as f:
#         chunk_data = json.load(f)
    
#     documents = []
#     for chunk in chunk_data:
#         documents.append(Document(page_content=chunk["page_content"], metadata=chunk["metadata"]))
    
#     print(f"Chunks loaded from {path}")
#     return documents


### pdf 로더

In [7]:
# def get_embedding():
#     device = 'cuda' if torch.cuda.is_available() and 'mps' in torch.cuda.get_device_capability() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
#     embeddings = HuggingFaceEmbeddings(
#         model_name='jhgan/ko-sroberta-multitask',
#         model_kwargs={'device': device},
#         encode_kwargs={'normalize_embeddings': True})
#     return embeddings

# def normalize_string(s):
#     try:
#         normalized = unicodedata.normalize('NFC', s)
#         return normalized.encode('utf-8', errors='replace').decode('utf-8')
#     except Exception:
#         return s

# def clean_text(text):
#     text = text.replace("�", "").replace("", "")  # 잘못된 인코딩 문자 제거
#     text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
#     return text

# def extract_text_and_tables(pdf_path):
#     text_content = ""
#     try:
#         # fitz를 사용한 텍스트 추출
#         doc = fitz.open(pdf_path)
#         for page in doc:
#             text = page.get_text("text")
#             if text:
#                 text = clean_text(normalize_string(text))
#                 text_content += text + "\n"

#         # pdfplumber를 사용한 표 추출
#         with pdfplumber.open(pdf_path) as pdf:
#             for page in pdf.pages:
#                 tables = page.extract_tables()
#                 if tables:
#                     text_content += "\n__table__\n"
#                     for table in tables:
#                         if table:
#                             for row in table:
#                                 processed_row = []
#                                 for cell in row:
#                                     if cell is None:
#                                         processed_row.append(" ")  # 병합된 셀 처리
#                                     else:
#                                         cell_text = normalize_string(clean_text(str(cell).strip()))
#                                         if "\n" in cell_text:
#                                             cell_text = " ".join(cell_text.split())
#                                         processed_row.append(cell_text)
#                                 row_text = " | ".join(processed_row)
#                                 text_content += row_text + "\n"
#                     text_content += "__end_table__\n"
#     except Exception as e:
#         print(f"Error extracting text and tables from {pdf_path}: {e}")
#     return text_content

# class CustomPDFLoader:
#     def __init__(self, file_path):
#         self.file_path = file_path

#     def load(self):
#         documents = []
#         try:
#             # 텍스트와 표를 모두 추출
#             text_content = extract_text_and_tables(self.file_path)

#             # 메타데이터 생성 및 Document 객체로 변환
#             metadata = {
#                 "source": self.file_path,
#             }
#             documents.append(Document(page_content=text_content, metadata=metadata))
#         except Exception as e:
#             print(f"Error loading PDF file {self.file_path}: {e}")
#         return documents

# def chunk_documents(documents):
#     adjusted_documents = []
#     for doc in documents:
#         content = doc.page_content
#         chunks = []
        
#         # 표 구분자를 기준으로 분할
#         table_sections = content.split("__table__")
        
#         for section in table_sections:
#             if "__end_table__" in section:
#                 table_content = section.split("__end_table__")[0]
#                 chunks.append(table_content)
#             else:
#                 chunk_splitter = RecursiveCharacterTextSplitter(
#                     chunk_size=1000,  # 텍스트 청크 크기를 키움
#                     chunk_overlap=200,
#                     separators=["\n\n", ".", "?", "!", "\n"]
#                 )
#                 normal_chunks = chunk_splitter.split_text(section)
#                 chunks.extend(normal_chunks)

#         for chunk in chunks:
#             adjusted_documents.append(Document(page_content=chunk, metadata=doc.metadata))
    
#     return adjusted_documents

# def format_docs(docs):
#     context = ""
#     for doc in docs:
#         context += doc.page_content
#         context += '\n\n'
#     return context

# def make_db(df):
#     documents = []
#     pdf_files = df['Source_path'].unique()

#     # tqdm으로 파일 처리 진행 상황 표시
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(load_pdf_file, pdf_files), total=len(pdf_files), desc="Processing PDFs"))
    
#     for result in results:
#         documents.extend(result)
    
#     # 정규화
#     for doc in documents:
#         doc.page_content = normalize_string(doc.page_content)
    
#     # 표가 망가지지 않도록 청크 분할 처리
#     chunks = chunk_documents(documents)
#     print(f"Total number of chunks: {len(chunks)}")
    
#     # FAISS DB 만들기 (메타데이터 포함)
#     faiss = FAISS.from_documents(chunks, embedding=get_embedding())
#     bm = KiwiBM25Retriever.from_documents(chunks)
    
#     return faiss, bm

# def load_pdf_file(pdf_file):
#     try:
#         pdf_loader = CustomPDFLoader(pdf_file)
#         return pdf_loader.load()
#     except Exception as e:
#         print(f"Skipping corrupted file {pdf_file}: {e}")
#         return []


### ..

In [4]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
# path_list = test_df['Source_path'].unique()
# documents=[]
# for i in range(len(path_list)):
#     print(path_list[i])
#     try:
#         pdf = pymupdf4llm.to_markdown(path_list[i],page_chunks=True)
#         total_pages = len(pdf)
        
#         for page_data in pdf:
        
#             page_number = page_data.get('metadata', {}).get('page', None)
#             if page_number is None:
#                 raise ValueError("Page number missing in metadata")

#             text = clean_text(normalize_string(page_data.get('text', '')))
#             if not text:
#                 raise ValueError("Empty text")  # 텍스트가 비어 있으면 예외 발생

#             metadata = {
#                 "Source_path": path_list[i],
#                 "page_number": page_number,
#                 "total_pages": total_pages,
#                 "tables": page_data.get('tables', None),
#             }

#             document = Document(page_content=text, metadata=metadata)
#             documents.append(document)
            
#     except:
#         print('error')
#         continue


./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
./test_source/보건복지부_부모급여(영아수당) 지원.pdf
./test_source/보건복지부_노인장기요양보험 사업운영.pdf
./test_source/산업통상자원부_에너지바우처.pdf
./test_source/국토교통부_행복주택출자.pdf
./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf
./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf
./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf
./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf


In [10]:
# print(documents[0].metadata)

# print(documents[0].page_content)

{'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf', 'page_number': 1, 'total_pages': 3, 'tables': [{'bbox': (58.27280044555664, 182.08099365234375, 535.1358032226562, 250.64599609375), 'rows': 3, 'columns': 7}, {'bbox': (58.27280044555664, 260.83502197265625, 535.1358032226562, 316.6939697265625), 'rows': 3, 'columns': 4}, {'bbox': (58.27280044555664, 368.8370056152344, 535.1358032226562, 400.48199462890625), 'rows': 2, 'columns': 7}, {'bbox': (58.27280044555664, 467.2489929199219, 535.1358032226562, 539.530029296875), 'rows': 3, 'columns': 7}]}
## 사  업  명

 혁신창업사업화자금(융자) (5152-301)


## 1. 사업 코드 정보

|구분|기금|소관|실국(기관)|계정|분야|부문|
|---|---|---|---|---|---|---|
|코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118|
|명칭|||||산업·중소기업 및 에너지|창업 및 벤처|


|구분|프로그램|단위사업|세부사업|
|---|---|---|---|
|코드|5100|5152|301|
|명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|


## 구분 기금 소관 실국(기관) 계정 분야 부문
 코드 110 118
중소벤처기업

중소벤처기업부 중소기업정책실 산업·중소기업

## 명칭 창업 및 진흥기금 창업 및 벤처

및 에너지

## 구분 프로그램 단위사업 세부사업
 코드 5100 5152 301
 명칭 창업

In [6]:
# faiss = FAISS.from_documents(documents, embedding=get_embedding())

/Users/seungwoo/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [25]:
# # 예를 들어, 메타데이터가 "category": "tech" 인 문서만 필터링하고 싶을 때
# docs = faiss.similarity_search(
#     query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
#     filter={"Source_path": test_df.iloc[34]['Source_path']},
#     k = 99,
#     fetch_k = 20000
# )
# docs

[Document(metadata={'Source_path': './test_source/산업통상자원부_에너지바우처.pdf', 'page_number': 3, 'total_pages': 11, 'tables': [{'bbox': (58.00899887084961, 77.91497802734375, 537.1510009765625, 371.4739990234375), 'rows': 4, 'columns': 14}]}, page_content='|Col1|2022|Col3|Col4|Col5|Col6|2023(’23.12월말)|Col8|Col9|Col10|Col11|Col12|Col13|2024 예산|\n|---|---|---|---|---|---|---|---|---|---|---|---|---|---|\n||예산액 (추경)|예산 현액|집행액 [실집 행액]|이월액|불용액|본예산|예산 현액|집행액 [실집 행액]|전년도 이월액 제외||이월 예상액|불용 예상액||\n||||||||||예산 현액|집행액 [실집 행액]||||\n|- 민간경상보조 (320-01) ․등유바우처 - 민간경상보조 (320-01) ․사업운영비 - 민간경상보조 (320-01) ․등유‧LPG지원금 - 민간경상보조 (320-01)|23,600 1,674 1,674 1,872 1,872 - -|23,600 1,674 1,674 1,872 1,872 - -|23,600 [22,304] 1,674 [1,674] 1,674 [1,674] 1,872 [1,808] 1,872 [1,808] - -|- - - - - - -|- - - - - - -|21,712 1,395 1,395 1,349 1,349 - -|23,854 1,551 1,551 3,485 3,485 74,964 74,964|23,854 [15,842] 1,551 [522] 1,551 [522] 3,485 [1,086] 3,485 [1,086] 74,964 [60,854] 74,964 [60,854]|23,854 1,551 1,551 3,485 3,48

In [20]:
# documents = []
# i =1
# querry = test_df.iloc[i]['Question']
# source_path= test_df.iloc[i]['Source_path']
# if source_path == test_df.iloc[i]['Source_path']:
#     documents = get_docs(source_path)
#     vectordb = FAISS.from_documents(documents, embedding=get_embedding())

#     bm = OktBM25Retriever.from_documents(documents)
#     knn = KNNRetriever.from_documents(documents, embeddings=get_embedding())
#     weight = [0.4, 0.3,0.3]
#     k = 2
#     dense = vectordb.as_retriever(
#         search_type = 'mmr',
#         search_kwargs={'k':k} )
#     bm.k = k
#     knn.k = k
#     retriever = EnsembleRetriever(retrievers=[dense, bm, knn], weight=weight)
# context = retriever.invoke(querry)
# for i in context:
#     print(i.page_content)
#     print(len(i.page_content))
#     print(i.metadata)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ㅇ 사업시행주체 : 중소벤처기업진흥공단

ㅇ 사업 수혜자 : 중소기업

ㅇ 보조, 융자, 출연, 출자 등의 경우 보조·융자 등 지원 비율 및 법적근거

|내역사업명|구분|피보조․피출연 등 기관명|지원 금액 (2024 계획안)|지원 비율(%)|보조율 법적근거 (해당 조항)|
|---|---|---|---|---|---|
|창업기반지원|융자|중소기업|1조 7,958억원|정책자금기준금리△0.3%p|중소기업진흥에 관한 법률 제66조, 제66조, 제67조, 제74조|
|개발기술사업화|융자 +보조|중소기업|2,120억원|(융자)정책자금기준금리 (이차보전) 최대 5.5% 이내|중소기업진흥에 관한 법률 제66조, 제66조, 제67조, 제74조|



## 7. 사업 집행절차

사업계획수립/공고 서류 및 현장실사

⇨ [사전상담 및 신청][‧][접수] ⇨

[‧]

(중기부, 중진공) (중소기업 → 중진공) (중진공 → 중소기업)


사업계획수립/공고

(중기부, 중진공)


융자 실행 지원결정통보 평가 및 승인

⇦

(중진공, 은행 → 중소기업) [⇦] (중진공 → 중소기업) (중진공 → 중소기업)


융자 실행

(중진공, 은행 → 중소기업)


-----


595
{'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf', 'page_number': 3, 'total_pages': 3, 'tables': [{'bbox': (70.84200286865234, 133.29400634765625, 536.9110107421875, 292.84002685546875), 'rows': 3, 'columns': 6}]}
## 사  업  명

 혁신창업사업화자금(융자) (5152-301)


## 1. 사업 코드 정보

|구분|기금|소관|실국(기관)|계정|분야|부문|
|---|---|---|---|---|---|---|
|코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118|
|명칭|||||산업·중소기업 및 

In [11]:
# markdown_headers = [
#     ("#", "Header 1"),
#     ("##", "Header 2"),
#     ("###", "Header 3"),
#     ("####", "Header 4"),
#     ("#####", "Header 5")
# ]

# markdown_splitter = MarkdownTextSplitter(
#     chunk_size=800,
#     chunk_overlap=60,
# )

# for doc in documents:
#     text = doc.page_content
#     metadata = doc.metadata
#     print(len(text))
#     md_header_splits = markdown_splitter.split_text(text)
#     for md_split in md_header_splits:
        
#         print(metadata)
#         print(len(md_split))
#         print(md_split)

        
    

In [12]:
# train_faiss_db,train_knn_retriever, train_bm_retriever = make_db(train_df)

100%|██████████| 16/16 [02:55<00:00, 10.94s/it]
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Total number of documents: 1078


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
# import textwrap
# from IPython.display import display
# from IPython.display import Markdown
# def to_markdown(text):
#   text = text.replace('•', '  *')
#   return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# def format_docs(docs):
#     context = ""
#     i = 1
#     for doc in docs:
#         context += f"Document: {i}" +"Source:"+ doc.metadata['Source_path'] +'\n'
#         context += doc.page_content
#         context += '\n---\n'
#         i += 1
#     return context

# def equal_path(contexts, source_path):
#     adjusted_docs = []
#     for doc in contexts:
#         if doc.metadata['Source_path'] == source_path:
#             adjusted_docs.append(doc)
#     return adjusted_docs
# i= 1

# weight = [0.4,0.3, 0.3]
# train_k = 3
# train_bm_retrievier.k = train_k
# train_knn_retriever.k = train_k
# train_faiss_retriever = train_faiss_db.as_retriever(
#     search_type = 'mmr',
#     search_kwargs={'k':train_k} )
# train_ensemble_retriever = EnsembleRetriever(
#     retrievers=[train_faiss_retriever, train_knn_retriever,train_bm_retrievier], weights=weight ,  
# )

# query = train_df.iloc[i]['Question']
# print(query)
# print(train_df.iloc[i]['Source_path'])
# results = train_ensemble_retriever.invoke(normalize_string(query))
# results = equal_path(results, train_df.iloc[i]['Source_path'])
# print(len(results))
# for result in results:
#     print(result.metadata)
#     print(result.page_content)
#     print('-------------------')

2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?
./train_source/1-1 2024 주요 재정통계 1권.pdf
4
{'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'page_number': 15, 'total_pages': 137, 'tables': []}
2024 주요 재정통계

### 한 눈에 보는 재정통계


-----


-------------------
{'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'page_number': 57, 'total_pages': 137, 'tables': []}
2024 주요 재정통계 | 2024 Fiscal Statistics

▸
2024년 기준, 62개 중앙관서의 소관별 지출은 보건복지부 122.4조원(18.6%), 교육부
95.8조원(14.6%), 행정안전부 72.4조원(11.0%), 국토교통부 60.9조원(9.3%) 등의 순임


5.
소관별



[표 3-13] 중앙관서별 최근 5년 총지출 추이 (단위: 억원,%)


2020
결산


2021
결산


2022
결산


2023
예산


2024
예산


전년 대비
증가율


884,437 941,640 1,053,939 1,091,830 1,223,779
보건복지부 12.1
(16.1) (15.7) (15.7) (17.1) (18.6)

758,407 827,479 1,005,384 1,019,979 957,888
교육부 -6.1
(13.8) (13.8) (15.0) (16.0) (14.6)

689,492 724,411 809,762 804,878 724,474
행정안전부 -10.0
(12.5) (12.1) (12.1) (12.6) (11.0)

493,251 569,481 553,445 557,514 609,439
국토교통부 9.3
(9.0) (9.5) (8.3) (8.7) (9.3)

337,759 363,751 381,3

In [38]:
faiss = make_db(test_df)

100%|██████████| 9/9 [00:08<00:00,  1.09it/s]


Total number of documents: 69


In [45]:
i = 61
k = 1
docs = faiss.similarity_search(
    query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
    filter={"Source_path": test_df.iloc[i]['Source_path']},
    k = 99,
    fetch_k = 20000
)
buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": 2})
knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
knn_retriever.k = k
bm_retriever = OktBM25Retriever.from_documents(docs)
bm_retriever.k = k

ensenble_retriever = EnsembleRetriever(retrievers=[faiss_retriever, knn_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])

print(test_df.iloc[i]['Question'])
print(test_df.iloc[i]['Source_path'])

# results_bm = bm_retriever.invoke(test_df.iloc[i]['Question'])
# results_faiss = faiss_retriever.invoke(test_df.iloc[i]['Question'])
# for i in results_bm:
#     print(i.metadata)
#     print('-------------------')
# print('============================================')
# for i in results_faiss:
#     print(i.metadata)
#     print('-------------------')
# print('============================================')
results = ensenble_retriever.invoke(test_df.iloc[i]['Question'])
for i in results:
    print(i.metadata)
    print(i.page_content)
    print('-------------------')
print(len(results))

재정성과관리는 무엇을 목표로 하는가?
./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf
{'Source_path': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page_number': 3, 'total_pages': 9, 'tables': []}
**FIS** **ISSUE & FOCUS**

-  사업을 임무와 전략 체계에 따라 구조화하고, 프로그램, 단위사업별 성과지표를 구비하여 재정 투입 성과를

직관적으로 알기 쉽도록 체계화

-  따라서 재정성과관리 체계는 사전 목표에 대한 사후 측정 및 평가, 유사한 사업 묶음 간의 표준화된 비교를

위한 재정사업 평가제도의 발전을 함께 수반

‣ 재정성과관리는 예산의 편성에서 집행, 결산에 이르는 전 과정에서 수행

-  재정성과관리는 예산 편성 과정에서 성과계획서를 작성하면서 시작하며, 이를 바탕으로 연간 성과지표와

목표치를 관리하여 국회에 보고하고 국민에게 공개

-  각 부처는 성과계획서·보고서 외에 재정사업에 대한 자율평가를 수행하며, 재정당국인 기획재정부는 지표가

적정한지 점검하고 개별 재정사업에 대한 평가를 수행할 수 있음

‣ 성과관리의 목적은 정부 재정의 투명성·책임성, 효율성·효과성, 예산재분배 등이 있음

-  사업 전략체계별 재원 배분, 성과목표와 달성도, 주요 수혜자에 대한 정보를 집약적으로 제공하여 투명성 및

책임성 향상

-  연간 성과관리 결과 도출된 문제점 등을 사업 관리에 환류(managerial feedback)하여 사업 성과를 제고하고

효과성을 향상

-  정책 우선순위, 더 효과적인 사업·정책으로 재원을 재배분함으로써 배분적 효율성 향상

**재정성과관리제도의 의의**

‣ 재정성과관리제도는 프로그램 단위의 성과관리, 사업 또는 사업군별로 수행되는 평가를 통해 거시적

재정총량 위주의 관리와 개별 부처의 재정 최하단부의 재정사업 관리체계를 연결해줌

-  재정사업에

In [16]:
total = []
for i in range(len(test_df)):
    query = test_df.iloc[i]['Question']
    results = test_ensemble_retriever.invoke(query)
    chunk_sizes = [len(i.page_content) for i in results]
    total.extend(chunk_sizes)

In [17]:
import numpy as np
np.mean(total)

1875.1284403669724

In [18]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def format_docs(docs):
    context = ""
    i = 1
    for doc in docs:
        context += f"Document: {i}" +"Source:"+ doc.metadata['file_path'] +'\n'
        context += doc.page_content
        context += '\n---\n'
        i += 1
    return context


i= 35

query = test_df.iloc[i]['Question']
print(query)
print(test_df.iloc[i]['Source_path'])
results = test_ensemble_retriever.invoke(normalize_string(query))
def equal_path(contexts, source_path):
    adjusted_docs = []
    for doc in contexts:
        if doc.metadata['file_path'] == source_path:
            adjusted_docs.append(doc)
    return adjusted_docs
results = equal_path(results, test_df.iloc[i]['Source_path'])
for result in  results:
    print(result.metadata)
    print(result.page_content)
    print('-------------------')


21년 국정감사에서 에너지 바우처 사업에 대한 주요 지적사항은 무엇이었나요?
./test_source/산업통상자원부_에너지바우처.pdf


KeyError: 'file_path'

In [ ]:
import warnings

# 특정 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore", message="No relevant docs were retrieved using the relevance score threshold")


rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0}

for i , entry in enumerate(train_df.to_dict(orient='records')):
    # print(entry['Question'])
    # print(entry['Source_path'])
    train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
    rag = train_retriever.invoke(entry['Question'])

    num_rag = len(rag)
    if num_rag ==0 :
        print(i)
        print(entry['Question'])
        print(entry['Source_path'])
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

NameError: name 'train_db' is not defined

In [ ]:
rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
for i , entry in enumerate(test_df.to_dict(orient='records')):
    num_rag = len(test_retriver.invoke(entry['Question']))
    if num_rag ==0 :
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

{'0': 9, '1': 12, '2': 77, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}


In [ ]:
i = 26
querry = test_df.iloc[i]['Question']
test_retriver = test_db.as_retriever(search_kwargs={'k': 3})
print("Q: ",querry)
len(test_retriver.invoke(querry))
# print(doc_list[0].page_content)
# print(doc_list[0].metadata)

Q:  하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


3

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'search_tpye':'ngram','k': 3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액')]

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, 'score_threshold': 0.8, "k":3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
test_db.similarity_search_with_relevance_scores(querry, k=10, score_threshold=0.8,filter={'source':test_df.iloc[i]['Source_path']})


[(Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
  0.8090853551046263)]

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, "k":3})
print(querry)
test_retriver.invoke(querry)

하절기바우처와 동절기바우처의 2024년 예산 규모는 각각 얼마인가요?


[Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액'),
 Document(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='’23년 예산 ’24년 예산\n예산 산출내역 예산 산출내역\n 190,963○ 민간경상보조 (320-01 ) : 190,963 백만원\n가. 하절기바우처 (36,851 백만원 )\n ㆍ 85.7만 세대 × 4.3만원 = 36,851 백만원\n나. 동절기바우처 (129,656 백만원 )\n ㆍ 85.3만 세대 × 15.2만원 = 129,656 백만원\n다. 연탄쿠폰 (21,712 백만원 )\n ㆍ 4.6만 세대 × 47.2만원 = 21,712 백만원\n라. 등유바우처 (1,395 백만원 )\n ㆍ 0.45만 세대 × 31만원 = 1,3

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = get_embedding()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.80)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=test_retriver
)
compression_retriever.invoke(querry)

[_DocumentWithState(metadata={'source': './test_source/산업통상자원부_에너지바우처.pdf', 'page': 5}, page_content='3)’24년도 예산안 산출 근거\n① 하절기바우처 : (‘23) 36,851 백만원 → (’24) 60,950 백만원 (+24,099 백만원 )\n   *  지원대상 확대(생계·의료→생계·의료·주거·교육)와 전기요금 인상에 따른 증액\n  - (산출) 85.7만세대 x 4.3만원(‘23년) → 115.0만세대 × 5.3만원(’24년)\n② 동절기바우처 : (‘23) 129,656 백만원 → (’24) 600,521 백만원 (+470,865 백만원 )\n   *  지원대상 증가와 난방비 인상에 따른 단가 인상 등 지원규모 확대를 고려한 증액\n  - (산출) 85.3만 세대 x 15.2만원(‘23년) → ① ‘23년 동절기 지원확대 : 113.1만 세대 x 30.4만원 x 70%, \n② ‘24년 동절기 지원 : 114.6만 세대 x 31.4만원(’24년)\n③ 연탄쿠폰 : (‘23) 21,712 백만원 → (’24) 21,240 백만원 (△472백만원 )\n   * 연탄 소비가구 감소 추세에 따른 감액', state={'embedded_doc': [0.039121244102716446, 0.012639015913009644, -0.011476924642920494, -0.02020144648849964, 0.03877473622560501, -0.05134795606136322, -0.01866687275469303, 0.06812097877264023, 0.05506658926606178, -0.009361985139548779, 0.055208753794431686, -0.010476823896169662, -0.05019139125943184, 0.0033026474993675947, -0.05579404532909393, -0.0050392174161970615, -0.01